In [1]:
import os
import pandas as pd
import datetime

In [2]:

positive = pd.read_excel('/Users/a2/Downloads/Hackathon2/41591_2021_1593_MOESM3_ESM.xlsx')
positive = positive.drop('Device', axis=1)
df = positive.copy()

In [3]:
base_folder = '/Users/a2/Downloads/Hackathon2/Phase2_Data'
pid_folders = os.listdir(base_folder)
for f in pid_folders:
    if f not in positive['Participant ID'].values:
        os.system(f"rm -rf {base_folder}/{f}")

In [4]:
def get_hr_st(id, date):
    files = os.listdir(f'/Users/a2/Downloads/Hackathon2/Phase2_Data/{id}')
    files.sort()
    temp = pd.read_csv(f'/Users/a2/Downloads/Hackathon2/Phase2_Data/{id}/{files[0]}')
    heartrate = temp.copy()
    base = 	heartrate.iloc[0]['datetime']
    summe = 0
    anzahl = 0
    heartrate_scaled = pd.DataFrame()


    for n, d in enumerate(heartrate['datetime']):
        if str(d)[:-3] == base[:-3]:
            summe += heartrate['heartrate'][n]
            anzahl += 1
        else:
            temp = [[pd.to_datetime(f"{base[:-3]}:00"), int(summe/anzahl)]]
            base = d
            heartrate_scaled = heartrate_scaled.append(temp)
            summe = heartrate['heartrate'][n] 
            anzahl = 1
    heartrate_scaled.rename(columns={0:'datetime', 1:'heartrate', 2: 'steps'}, inplace=True)
    temp = pd.read_csv('/Users/a2/Downloads/Hackathon2/Phase2_Data/P110465/Orig_Fitbit_ST.csv')
    steps = temp.copy()
    count = 0
    steps_normalized = pd.DataFrame()
    for i in range(len(steps)-1): 
        value = (steps.iloc[i]['steps']+ steps.iloc[i+1]['steps'] )/ ((pd.to_datetime(steps.iloc[i+1]['datetime']) - pd.to_datetime(steps.iloc[i]['datetime'])).total_seconds()/60)/2
        while pd.to_datetime(heartrate_scaled.iloc[count]['datetime']) < pd.to_datetime(steps.iloc[i+1]['datetime']) :
            steps_normalized = steps_normalized.append([value])
            count = count + 1
            try:
                assert(count<len(heartrate_scaled))
            except BaseException:
                break
    heartrate_scaled['steps'] = steps_normalized
    date = pd.DateOffset(days=4)
    inf_date = pd.DataFrame()

    for i in range(len(heartrate_scaled)):
        if inf_date < heartrate_scaled['datetime'][i]:
            inf_date.append([1])
        else:
            inf_date.append([0])
    heartrate_scaled['positive'] = inf_date

    # date ist 4 Tage zu spät


In [9]:
get_hr_st('P530788', 0)

KeyboardInterrupt: 

In [5]:
# 4 days 
for i in range(len(df)):
    hl = get_hr_st(df['Participant ID'][i], df['COVID-19 Test Date'][i])

    # print(df['Participant ID'][i])



/var/folders/nd/5lmcqchd27b_ctzb2whh36800000gn/T/ipykernel_54582/1672305811.py:41: FutureWarning: Automatic reindexing on DataFrame vs Series comparisons is deprecated and will raise ValueError in a future version.  Do `left, right = left.align(right, axis=1, copy=False)` before e.g. `left == right`
  if inf_date < heartrate_scaled['datetime'][i]:


TypeError: Invalid comparison between dtype=datetime64[ns] and ndarray